In [1]:
%pip install -q kaggle
%pip install -q kagglehub
%pip install -q dask
%pip install -q ipywidgets --upgrade

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd, requests
import kagglehub, re, json, os
import requests, subprocess
import sys, time
from tqdm import tqdm

In [6]:
# Définis le chemin du fichier kaggle.json
path = r"C:\Credentials\kaggle.json"
#json file : {"username":"flosrv","api_key":"44c79a4d5d1534092d24ee29f34e6ee3"}
# Charge les credentials depuis le fichier JSON
with open(path, 'r') as file:
    content = json.load(file)

# Récupère les valeurs du fichier
username = content["username"]
api_key = content["key"]  # ⚠️ Assure-toi que la clé s'appelle bien "key" dans ton fichier JSON

# ✅ Définis les variables d'environnement avant de les utiliser
os.environ['KAGGLE_USERNAME'] = username
os.environ['KAGGLE_KEY'] = api_key
import kaggle
from kaggle import api # import the already authenticated API client
from kaggle.api.kaggle_api_extended import KaggleApi
# 1. **Authentification avec Kaggle**
def authenticate_kaggle():
    # Vérifie si le fichier kaggle.json est dans le bon dossier
    kaggle_json_path = os.path.expanduser(path)  # pour Linux/macOS
    # Pour Windows, utilisez : os.path.expanduser(r'C:\Users\<Votre-Nom>\.kaggle\kaggle.json')

    # Si le fichier n'existe pas, avertir l'utilisateur
    if not os.path.exists(kaggle_json_path):
        print("Le fichier 'kaggle.json' n'est pas trouvé. Téléchargez-le depuis votre compte Kaggle.")
        print("Assurez-vous de le placer dans le répertoire ~/.kaggle/ (Linux/macOS) ou C:\\Users\\<Your-Username>\\.kaggle\\ (Windows).")
        return False  # L'authentification échoue
    else:
        print(f"Authentification réussie avec le fichier '{kaggle_json_path}'")
        api = KaggleApi()
        api.authenticate()  # Authentification avec le fichier 'kaggle.json'
        return True

# Exécuter l'authentification
authenticate_kaggle()

Authentification réussie avec le fichier 'C:\Credentials\kaggle.json'


True

In [ ]:
path = r"C:\Users\f.gionnane\Downloads\Mes-Projets\Kaggle Data Projects\data"

Functions For Datasets

In [ ]:
def list_datasets(keyword=None, usability_min=None, min_mb=None, columns=None):
    print(f"Lister les datasets correspondant à '{keyword if keyword else 'tous les datasets'}' :")
    
    api = KaggleApi()
    api.authenticate()

    datasets = api.datasets_list(search=keyword) if keyword else api.datasets_list()
    
    result = []
    for dataset in datasets:
        dataset_dict = dataset  

        if columns is not None:
            dataset_dict = {key: dataset[key] for key in dataset if key in columns}

        result.append(dataset_dict)
    
    df = pd.DataFrame(result)
    df = df.loc[:, ~df.columns.str.contains('has|Nullable', case=False)]

    if "tags" in df.columns:
        df["tags"] = df["tags"].apply(
            lambda x: [d["nameNullable"] for d in x if isinstance(d, dict) and "nameNullable" in d] if isinstance(x, list) else []
        )

    if "usabilityRating" in df.columns:
        df["usabilityRating"] = round(df["usabilityRating"], 2)

    if usability_min is not None and "usabilityRating" in df.columns:
        df = df[df["usabilityRating"] >= usability_min]

    if min_mb is not None and "Size (Mb)" in df.columns:
        df = df[df["Size (Mb)"] >= min_mb]

    if keyword:
        df = df[
            df["title"].str.contains(keyword, case=False, na=False) |  
            df["tags"].apply(lambda tags: any(keyword in tag.lower() for tag in tags))
        ]

    if "url" in df.columns:
        for idx, row in df.iterrows():
            try:
                response = requests.get(row["url"])
                if response.status_code >= 400:
                    df.drop(idx, inplace=True)
            except requests.exceptions.RequestException:
                df.drop(idx, inplace=True)

    for idx, row in df.iterrows():
        ref = row["ref"]
        title = row["title"]
        
        response = requests.get(f"https://www.kaggle.com/api/v1/datasets/metadata/{ref}")
        
        if response.status_code == 200:
            content = response.json()  
            
            if content["info"]["isPrivate"]:
                df.drop(labels=idx, inplace=True)  
        
        else:
            print(f"Error fetching metadata for {title} (Status code: {response.status_code})")

    if columns is not None:
        df = df[[col for col in columns if col in df.columns]]  # Réorganiser les colonnes dans l'ordre de la liste fournie

    if "totalBytes" in df.columns:
        df["totalBytes"] = round(df["totalBytes"] / 1_000_000, 2)
        df.rename(columns={"totalBytes": "Size (Mb)"}, inplace=True)
    
    if "usabilityRating" in df.columns:
        df.rename(columns={"usabilityRating": "Usability"}, inplace=True)
    

    print(f'{df.shape[0]} résultat(s) trouvé(s) ! ')
    return df


def download_dataset(dataset_ref: str, path: str =path):
    """
    Télécharge un dataset Kaggle via l'API en créant un dossier pour chaque dataset, avec une barre de progression.

    Args:
        dataset_ref (str): Référence du dataset sous la forme "username/dataset-name".
        path (str, optionnel): Répertoire où enregistrer le fichier téléchargé. Par défaut: ".".

    Returns:
        None
    """
    dataset_identifier = dataset_ref
    # Création du dossier pour le dataset avec son nom
    dataset_folder = os.path.join(path, dataset_identifier.replace("/", "_"))

    # Si le dossier existe déjà, informer l'utilisateur
    if os.path.exists(dataset_folder):
        print(f"✅ Le dataset '{dataset_identifier}' est déjà téléchargé dans '{dataset_folder}'.")
        return

    # Créer un dossier pour le dataset
    os.makedirs(dataset_folder)
    print(f"📂 Création du dossier pour le dataset '{dataset_identifier}' dans '{dataset_folder}'...")

    # Récupérer la taille du dataset via l'API Kaggle
    metadata_url = f"https://www.kaggle.com/api/v1/datasets/metadata/{dataset_identifier}"
    try:
        response = requests.get(metadata_url)
        if response.status_code == 200:
            metadata = response.json()
            dataset_size = metadata["Size (Mb)"] / 1_000_000  # Conversion en Mo
            print(f"📥 Téléchargement de '{dataset_identifier}' ({dataset_size:.2f} MB)...")
        else:
            print("⚠️ Impossible de récupérer la taille du dataset.")
            dataset_size = None
    except requests.exceptions.RequestException:
        print("⚠️ Erreur lors de la récupération des métadonnées.")
        dataset_size = None

    # Commande de téléchargement
    command = ["kaggle", "datasets", "download", "-d", dataset_identifier, "-p", dataset_folder, "--unzip"]

    try:
        # Démarrer le téléchargement avec suivi de progression
        process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Barre de progression
        with tqdm(total=dataset_size, unit="MB", unit_scale=True, desc="📥 Progression") as pbar:
            while process.poll() is None:
                time.sleep(1)
                pbar.update(1)

        # Vérifier si le téléchargement a réussi
        if process.returncode == 0:
            print(f"✅ Dataset '{dataset_identifier}' téléchargé avec succès dans '{dataset_folder}'.")
        else:
            print(f"❌ Erreur lors du téléchargement. Code de retour: {process.returncode}")

    except subprocess.CalledProcessError as e:
        print(f"❌ Erreur dans la commande Kaggle. Code de retour: {e.returncode}")
        print(f"Erreur : {e.stderr.decode('utf-8')}")
    except FileNotFoundError:
        print("❌ Erreur : La commande 'kaggle' n'a pas été trouvée. Installez l'API Kaggle.")
    except Exception as e:
        print(f"❌ Une erreur inattendue s'est produite : {str(e)}")

In [ ]:
columns= ['id',  'title', 'ref', 'totalBytes', 'usabilityRating', 
          'description', 'url', 'lastUpdated', 'topicCount', 
          'voteCount', 'currentVersionNumber', 'tags',
          'isPrivate']

In [ ]:
df_ocean_datasets =list_datasets(keyword='finance', usability_min=0.6, min_mb=5,columns=columns)
df_ocean_datasets.head()

In [ ]:
print(df_ocean_datasets.columns)
df_ocean_datasets.shape

In [ ]:
for idx, row in df_ocean_datasets.iterrows():

    ref = row["ref"]
    download_dataset(dataset_ref=ref, path=path)

In [ ]:
url_list = []
metadata_list = []
count = 0

for idx, row in df_ocean_datasets.iterrows():
    url = row["url"]
    url_list.append(url)
    ref = row["ref"]
    
    response = requests.get(f"https://www.kaggle.com/api/v1/datasets/metadata/{ref}")
    
    if response.status_code == 200:
        content = response.json()  # Stocker le JSON
        metadata_list.append(content)  # Ajouter à la liste des métadonnées
        
        # Vérifier si le dataset est privé
        if content["info"]["isPrivate"]:
            df_ocean_datasets.drop(labels=idx, inplace=True)  # Supprimer la ligne
            count += 1
            
    else:
        print(f"Error fetching metadata for {url} (Status code: {response.status_code})")

df_metadata = pd.DataFrame(metadata_list)
print(f"Nombre de datasets privés supprimés : {count}")
df_metadata.head()

In [ ]:
# Normaliser la colonne "info" avant d'appliquer le filtrage des colonnes
df_normalized = df_metadata.join(pd.json_normalize(df_metadata["info"]))
df_normalized.drop(columns=["info"], inplace=True)

# Appliquer le filtrage des colonnes après la normalisation
df_normalized = df_normalized.loc[:, ~df_normalized.columns.str.contains('has|nullable', case=False)]

df_normalized.head()

In [ ]:
print(df_normalized.shape)
df_normalized["isPrivate"].value_counts()

Functions For Models

In [ ]:
def list_models(keyword):
    print(f"Lister les modèles correspondant à '{keyword}' :")
    
    api = KaggleApi()
    api.authenticate()
    
    models = api.models_list(search=keyword)
    desired_columns = [
        'id', 'ref', 'title', 'description', 'url',
        'totalVotes', 'lastUpdated', 'tags', 'licenseName'
    ]
    
    result = []
    for model in models:
        model_dict = {key: model[key] for key in model if key in desired_columns}
        result.append(model_dict)
    
    df = pd.DataFrame(result)

    # Suppression des colonnes inutiles contenant "has" ou "Nullable"
    df = df.loc[:, ~df.columns.str.contains('has|Nullable', case=False)]
    df["tags"] = df["tags"].apply(lambda x: [d["name"] for d in x] if x is not None else [])
    # Vérification des liens URL pour voir si les modèles sont accessibles
    for idx, row in df.iterrows():
        url = row["url"]
        try:
            response = requests.get(url)
            if response.status_code >= 400:
                df.drop(idx, inplace=True)
        except requests.exceptions.RequestException:
            df.drop(idx, inplace=True)
    

    return df

In [ ]:
url_list =[]

for idx, row in df_ocean_datasets.iterrows():
    url = row["url"]
    url_list.append(url)
    title = row["title"]
    if title == 'OCEAN DATA / CLIMATE CHANGE / NASA':
        try:
            download_dataset(url)
        except Exception as e:
            print(e)
    else :
        print('dataset not found')

In [ ]:
df_plus_80 = df_ocean_datasets[df_ocean_datasets["usabilityRating"]>0.8]

In [ ]:
df_plus_80.head()